In [1]:
from sentence_transformers import SentenceTransformer, util

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

df = pd.read_csv('data.csv')

question = df['Question']
answer = df['Answer']


In [3]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [4]:
embeddings = model.encode(question)

In [12]:
# Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

# Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim) - 1):
    for j in range(i + 1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

In [5]:
# Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

NameError: name 'all_sentence_combinations' is not defined

In [8]:
from FlagEmbedding import FlagReranker

reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)


In [11]:
import tkinter as tk

def change_text():
    user_question = entry.get()
    q_embedding = model.encode(user_question)

    cos_sim = util.cos_sim(q_embedding, embeddings)

    val = []

    for i in range(len(cos_sim[0])):
        val.append(cos_sim[0][i].item())

    p_answer = answer.copy()
    p_question = question.copy()

    for i in range(len(val)):
        for j in range(i + 1, len(val)):
            if (val[i] < val[j]):
                temp = val[i]
                val[i] = val[j]
                val[j] = temp

                temp = p_answer[i]
                p_answer[i] = p_answer[j]
                p_answer[j] = temp
                
                temp = p_question[i]
                p_question[i] = p_question[j]
                p_question[j] = temp

    if (reranker.compute_score([p_question[0], user_question]) > 0):
        answer_label.config(text=p_answer[0])
    else:
        answer_label.config(text="Sorry, we don't know the answer for this question")


window = tk.Tk()
window.geometry('520x300')
label = tk.Label(text="FAQ")
label.pack()

entry = tk.Entry(fg="black", bg="white", width=50)
entry.pack()

# Create a button
button = tk.Button(window, text="Find", command=change_text)
button.pack()

noti_label = tk.Label(text="Answer")
noti_label.pack()

answer_label = tk.Label(text="", wraplength=300, justify='center')
answer_label.pack()

# Run the main event loop
window.mainloop()